In [1]:
import pandas as pd
import spacy
from collections import Counter
import re 

Einlesen von der CSV und aufspalten auf Religion und Atheism

In [2]:
data = pd.read_csv("20 newsgroups/20newsgroups.csv", on_bad_lines='skip', sep=';')
data.dropna()
data = data.iloc[:,1:3]

dataSplit = data.drop(data[data["group"] == 1 ].index)
dataSplit = dataSplit.drop(data[data["group"] == 2 ].index)
dataAth = data.drop(data[data["group"] != 0 ].index)
dataReli = data.drop(data[data["group"] != 3 ].index)

dataSpace = data.drop(data[data["group"] != 2 ].index)
dataGraphics = data.drop(data[data["group"] != 1 ].index)
nlp = spacy.load("en_core_web_sm")

                        Freq_words Generierung                                                                        
Änderung zu vorherigem Code: 
- toke.text.strip entfernt leere oder Whitespace-only Einträge
- regex entfernt alle nicht Zahlen oder Buchstaben

In [ ]:
word_freqAth = Counter()
for text in dataAth["text"]:
    text = str(text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    doc = nlp(text)
    filtered_words = [token.text.lower() for token in doc if (not token.is_stop and not token.is_punct and token.text.strip())]
    word_freqAth.update(filtered_words)

In [4]:
word_freqReli = Counter()
for text in dataReli["text"]:
    text = str(text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    doc = nlp(text)
    filtered_words = [token.text.lower() for token in doc if (not token.is_stop and not token.is_punct and token.text.strip())]
    word_freqReli.update(filtered_words)

In [5]:
word_freqSpace = Counter()
for text in dataSpace["text"]:
    text = str(text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    doc = nlp(text)
    filtered_words = [token.text.lower() for token in doc if (not token.is_stop and not token.is_punct and token.text.strip())]
    word_freqSpace.update(filtered_words)

In [6]:
word_freqGraphics = Counter()
for text in dataGraphics["text"]:
    text = str(text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    doc = nlp(text)
    filtered_words = [token.text.lower() for token in doc if (not token.is_stop and not token.is_punct and token.text.strip())]
    word_freqGraphics.update(filtered_words)

Tests:

In [7]:
print(word_freqReli.most_common(2))
print(word_freqReli.get("nt"))

[('nt', 704), ('god', 586)]
704


Generierung der neuen most_common_words:
Notwendig sind für 500 Worte:
- Reli: 
- Ath: 
- Graphics:
- Space:

In [14]:
faktor = 1
faktorRA = 3
words = 700

freqList_Reli = []
for word, freq in word_freqReli.most_common(10000):
    freqAth = word_freqAth.get(word)
    freqSpace = word_freqSpace.get(word)
    freqGraphics = word_freqGraphics.get(word)
    if len(freqList_Reli) >= words:
        break
    if ((freqAth is None or freq >= faktorRA * freqAth) and 
        (freqSpace is None or freq >= faktorRA * freqSpace) and 
        (freqGraphics is None or freq >= faktorRA * freqGraphics)):
        freqList_Reli.append(word)


freqList_Ath = []
for word, freq in word_freqAth.most_common(10000):
    freqReli = word_freqReli.get(word)
    freqSpace = word_freqSpace.get(word)
    freqGraphics = word_freqGraphics.get(word)
    if len(freqList_Ath) >= words:
        break
    if ((freqReli is None or freq >=faktorRA * freqReli) and 
        (freqSpace is None or freq >= faktorRA * freqSpace) and 
        (freqGraphics is None or freq >= faktorRA * freqGraphics)):
        freqList_Ath.append(word)

freqList_Space = []
for word, freq in word_freqSpace.most_common(10000):
    freqAth = word_freqAth.get(word)
    freqReli = word_freqReli.get(word)
    freqGraphics = word_freqGraphics.get(word)
    if len(freqList_Space) >= words:
        break
    if ((freqAth is None or freq >= faktor * freqAth) and 
        (freqGraphics is None or freq >= faktor * freqGraphics) and 
        (freqReli is None or freq >= faktor * freqReli)):
        freqList_Space.append(word)

freqList_Graphics = []
for word, freq in word_freqGraphics.most_common(10000):
    freqAth = word_freqAth.get(word)
    freqReli = word_freqReli.get(word)
    freqSpace = word_freqSpace.get(word)
    if len(freqList_Graphics) >= words:
        break
    if ((freqAth is None or freq >= faktor * freqAth) and 
        (freqSpace is None or freq >= faktor * freqSpace) and 
        (freqReli is None or freq >= faktor * freqReli)):
        freqList_Graphics.append(word)

Datei anlegen und befüllen:

In [15]:
freqList = freqList_Ath+freqList_Reli+freqList_Graphics+freqList_Space+["groupID"]
freqListData = pd.DataFrame(columns=freqList)
freqListData.to_csv("20newsgroups_freqList.csv", index=False)
freqListData.head()

,atheists,argument,atheism,atheist,islam,exist,theism,quran,islamic,fallacy,...,increases,topic,forces,batse,consideration,physicist,distances,fahrenheit,relatively,groupID


In [13]:
for index, row in data.iterrows():
    text = str(row['text'])
    group = str(row['group'])
    doc = nlp(text)
    count_in_freqList = Counter(token.text.lower() for token in doc if token.is_alpha)

    haeufigkeiten = {word: count_in_freqList.get(word, 0) for word in freqList}
    haeufigkeiten['groupID'] = group
    # Erstelle ein neues DataFrame aus der Zeile der Häufigkeiten
    neue_zeile_df = pd.DataFrame([haeufigkeiten])
    neue_zeile_df.reset_index(drop=True, inplace=True)
    freqListData = pd.concat([freqListData, neue_zeile_df], ignore_index=True)

freqListData.to_csv("20newsgroups_freqList.csv", index=False)
freqListData.head()


InvalidIndexError: Reindexing only valid with uniquely valued Index objects